In [ ]:
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten # Import Flatten layer

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Load the model (adjust the path to where your model is saved)
model_path = '/content/drive/MyDrive/Plant_CNN_17class_Enhanced.h5'
model = tf.keras.models.load_model(model_path)
print("✅ Model loaded successfully!")

# Add Flatten layer if necessary (inspect model summary to confirm)
# Check the layer before the first Dense layer in model.summary()
# If it's a convolutional or pooling layer output, a Flatten layer is needed.
# You might need to rebuild the model's sequential structure to insert Flatten
# For example, if the model is sequential:
# new_model = tf.keras.Sequential()
# for layer in model.layers[:-1]: # Add all layers except the last one
#     new_model.add(layer)
# new_model.add(Flatten()) # Add Flatten layer
# new_model.add(model.layers[-1]) # Add the last (Dense) layer
# model = new_model # Use the new model

# A simpler way if the model is sequential is to insert the layer:
if isinstance(model, tf.keras.Sequential):
    # Find the index to insert the Flatten layer before the first Dense layer
    insert_index = None
    for i, layer in enumerate(model.layers):
        if isinstance(layer, tf.keras.layers.Dense):
            insert_index = i
            break
    if insert_index is not None and not any(isinstance(model.layers[j], Flatten) for j in range(insert_index)):
        # Create a new list of layers with Flatten inserted
        new_layers = model.layers[:insert_index] + [Flatten()] + model.layers[insert_index:]
        # Create a new Sequential model with the updated layers
        new_model = tf.keras.Sequential(new_layers)
        # Transfer weights (this might require careful handling depending on the layers)
        # A simpler approach might be to rebuild and reload weights if inserting is complex
        # For this specific error, adding Flatten is likely the issue in model architecture
        print("Attempting to add Flatten layer before the first Dense layer.")
        # Note: Directly inserting might not work depending on how the model was saved.
        # A safer approach if the above fails is to define the model structure
        # and load weights layer by layer, or modify the original script that built the model.

# Print model summary to inspect layers and shapes
model.summary()


# 3. Setup test dataset generator
test_dir = '/content/drive/MyDrive/PlantDataset/test'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),  # change if your model uses a different input size
    batch_size=32,
    class_mode='categorical',  # or 'binary' depending on your dataset
    shuffle=False
)

# 4. Evaluate the model
# Ensure the model is compiled after loading if necessary, although the warning suggests it was compiled.
# If evaluation still fails, you might need to recompile the model here.
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # Example compile

loss, acc = model.evaluate(test_generator)
print(f"📊 Test Accuracy: {acc*100:.2f}%")
print(f"📉 Test Loss: {loss:.4f}")

# 5. (Optional) Make predictions on test set
predictions = model.predict(test_generator)
print("🔮 Predictions shape:", predictions.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


✅ Model loaded successfully!


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 17)             │         4,369 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,521,299 (24.88 MB)

...

 Non-trainable params: 448 (1.75 KB)

 Optimizer params: 2 (12.00 B)

Found 1520 images belonging to 17 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


48/48 ━━━━━━━━━━━━━━━━━━━━ 561s 12s/step - accuracy: 0.7159 - loss: 0.7438
📊 Test Accuracy: 74.14%
📉 Test Loss: 0.6656
48/48 ━━━━━━━━━━━━━━━━━━━━ 21s 424ms/step
🔮 Predictions shape: (1520, 17)
